## **Técnicas de Imputación Datos INMET**

#### **Regla de Imputación Estacional**

Supongamos que tienes una serie de datos:

- $R_i$: valor de radiación en la observación $i$
- $d_i$: día del año de la observación $i$ (de 1 a 366)
- $h_i$: hora del día de la observación $i$ (de 0 a 23)

Para cada observación con valor faltante $R_i = 0$, la imputación es:

$$
\hat{R}_i = \frac{1}{n_{d_i,h_i}} \sum_{j \in G(d_i,h_i)} R_j
$$

**Donde:**

- $\hat{R}_i$ es el valor imputado para la observación $i$
- $G(d_i, h_i)$ es el conjunto de observaciones $j$ donde $R_j > 0$, y el día y la hora coinciden con $d_i, h_i$
- $n_{d_i,h_i}$ es el número de valores disponibles en ese grupo

> Si no existen datos en ese grupo:

$$
\hat{R}_i = 0
$$

Fuentes: https://www.sciencedirect.com/science/article/pii/S2213138821007785?ref=pdf_download&fr=RR-2&rr=9266d1c478c4a4cd 

 https://www.mdpi.com/2071-1050/15/13/10609 

_Identificar el patrón típico de radiación para cada hora del día usando valores históricos disponibles del mismo día y mes, y luego aplicar ese valor como estimación cuando hay datos faltantes_

#### **Imputación datos INMET por Regla de Imputación Estacional**

In [ ]:
df_comparado['RADIACAO_GLOBAL_IMPUTADA'] = df_comparado['RADIACAO_GLOBAL'].replace(0, pd.NA)
df_comparado['datetime'] = pd.to_datetime(df_comparado['datetime'])
df_comparado['dayofyear'] = df_comparado['datetime'].dt.dayofyear
df_comparado['hour'] = df_comparado['datetime'].dt.hour

tabla_promedios = df_comparado.groupby(['dayofyear', 'hour'])['RADIACAO_GLOBAL_IMPUTADA'].mean()
def imputar_estacional(row):
    if pd.isna(row['RADIACAO_GLOBAL_IMPUTADA']):
        return tabla_promedios.get((int(row['dayofyear']), int(row['hour'])), 0)
    else:
        return row['RADIACAO_GLOBAL_IMPUTADA']
df_comparado['RADIACAO_GLOBAL_IMPUTADA'] = df_comparado.apply(imputar_estacional, axis=1)
df_comparado['RADIACAO_GLOBAL_IMPUTADA'] = df_comparado['RADIACAO_GLOBAL_IMPUTADA'].fillna(0)
print(df_comparado.shape)
df_comparado.tail(20)

#### **Evolución de Radiación Solar Original vs Imputada (Promedio Estacional)**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['RADIACAO_GLOBAL_IMPUTADA'],
    mode='lines',
    name='INMET-IMPUTADA',
    line=dict(color='orange'),
    opacity=0.7
))
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['RADIACAO_GLOBAL'],
    mode='lines',
    name='INMET-SIN IMPUTAR',
    line=dict(color='blue'),
    opacity=0.7
))
fig.update_layout(
    title='Evolución de Radiación INMET - Imputación Promedio Estacional (2019-2025)',
    xaxis_title='Fecha y Hora',
    yaxis_title='Radiación (KJ/m²)',
    legend_title='Fuente',
    template='plotly_white',
    height=500,
    width=1100
)
fig.show()

#### **Distribución de Antes y Despues de la Imputación Promedio Estacional**

La comparación gráfica de las distribuciones antes y después de la imputación muestra que ambas series presentan estructuras similares en términos generales, aunque con ligeros ajustes en la densidad y forma de la distribución. Sin embargo, para verificar si estas diferencias son significativas desde el punto de vista estadístico, se recurre al valor \( p \) de la prueba KS.

In [ ]:
rad_orig = df_comparado['RADIACAO_GLOBAL'][df_comparado['RADIACAO_GLOBAL'] > 0]
rad_imp = df_comparado['RADIACAO_GLOBAL_IMPUTADA'][df_comparado['RADIACAO_GLOBAL_IMPUTADA'] > 0]

fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)

sns.histplot(rad_orig, bins=60, kde=True, color='royalblue', ax=axes[0])
axes[0].set_title('RADIACAO_GLOBAL (> 0)')
axes[0].set_xlabel('Radiación (KJ/m²)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True)

sns.histplot(rad_imp, bins=60, kde=True, color='darkorange', ax=axes[1])
axes[1].set_title('RADIACAO_GLOBAL_IMPUTADA (> 0)')
axes[1].set_xlabel('Radiación (KJ/m²)')
axes[1].set_ylabel('')
axes[1].grid(True)

fig.suptitle('Distribuciones de Radiación Solar INMET (Imputación Promedio Estacional) (> 0) - Antes y Después ', fontsize=16)
plt.tight_layout()
plt.show()

#### **Evaluación de Antes y Despues de la Imputación Promedio Estacional**

Para validar si esta imputación mantiene la integridad estadística de la serie original, se utilizó la **prueba de Kolmogórov–Smirnov (KS)**, la cual permite comparar si dos muestras provienen de la misma distribución sin asumir normalidad.

- Si el valor \( p \) es **mayor que 0.05**, **no se rechaza la hipótesis nula**, lo que indica que la distribución imputada puede considerarse **estadísticamente equivalente** a la original.  
- Por el contrario, si el valor \( p \) es **menor o igual que 0.05**, se concluye que la imputación **modificó significativamente** la distribución original.

Este enfoque permite **validar objetivamente la calidad de la imputación**, asegurando que el tratamiento de valores faltantes no haya introducido sesgos ni distorsionado la naturaleza estadística del conjunto de datos.

In [ ]:
rad_orig = df_comparado['RADIACAO_GLOBAL'][df_comparado['RADIACAO_GLOBAL'] > 0]
rad_imp = df_comparado['RADIACAO_GLOBAL_IMPUTADA'][df_comparado['RADIACAO_GLOBAL_IMPUTADA'] > 0]
stat_ks, p_ks = ks_2samp(rad_orig, rad_imp)
print("Kolmogorov-Smirnov Test (Distribución Original vs Imputada):")
print(f"Estadístico KS: {stat_ks:.4f}")
print(f"p-valor: {p_ks:.4e}")
if p_ks > 0.05:
    print("No se rechaza H0 → Las distribuciones son estadísticamente similares.")
else:
    print("Se rechaza H0 → Las distribuciones son diferentes.")